In [1]:
import json
from json import dumps
import pandas as pd
import numpy as np

In [2]:
df =  pd.read_json(r'horarios_20211_consolidado_limpio.json', encoding='utf-8')

In [3]:
df

,Cod,Dia,Docente,Edificio,Espacio_Academico,Grp,Hora,INSCRITOS,Proyecto_curricular,Salon,Sede
0,12,MARTES,MARIN DIAZ DORA LILIA,PALACIO LA MERCED,CATEDRA DEMOCRACIA Y CIUDADANIA,102-1,10-11,25,ARTE DANZARIO,AUDITORIO SAMUEL BEDOYA,ACADEMIA SUPERIOR ARTES-ASAB
1,12,MARTES,MARIN DIAZ DORA LILIA,PALACIO LA MERCED,CATEDRA DEMOCRACIA Y CIUDADANIA,102-1,11-12,25,ARTE DANZARIO,AUDITORIO SAMUEL BEDOYA,ACADEMIA SUPERIOR ARTES-ASAB
2,12,JUEVES,MARIN DIAZ DORA LILIA,PALACIO LA MERCED,CATEDRA DEMOCRACIA Y CIUDADANIA,102-2,16-17,22,ARTE DANZARIO,AUDITORIO SAMUEL BEDOYA,ACADEMIA SUPERIOR ARTES-ASAB
3,12,JUEVES,MARIN DIAZ DORA LILIA,PALACIO LA MERCED,CATEDRA DEMOCRACIA Y CIUDADANIA,102-2,17-18,22,ARTE DANZARIO,AUDITORIO SAMUEL BEDOYA,ACADEMIA SUPERIOR ARTES-ASAB
4,3002,MARTES,CARDENAS GARZON JOHN MARIO,CENTRO CULTURAL NUEVA SANTAFE,PRODUCCION Y COMPRENSION DE TEXTOS,102-1,10-11,20,ARTE DANZARIO,S. JUEGOS,CENTRO CULTURAL NUEVA SANTAFE
...,...,...,...,...,...,...,...,...,...,...,...
25422,9919,SABADO,POR ASIGNAR,ALEJANDRO SUAREZ COPETE,SEGUNDA LENGUA I - NATIVA 1,105-54,13-14,0,INSTITUTO DE LENGUAS UNIVERSIDAD DISTRITAL,AULA 308,CALLE 40
25423,9919,SABADO,POR ASIGNAR,ALEJANDRO SUAREZ COPETE,SEGUNDA LENGUA I - NATIVA 1,105-54,14-15,0,INSTITUTO DE LENGUAS UNIVERSIDAD DISTRITAL,AULA 308,CALLE 40
25424,9919,SABADO,POR ASIGNAR,ALEJANDRO SUAREZ COPETE,SEGUNDA LENGUA I - NATIVA 1,105-54,15-16,0,INSTITUTO DE LENGUAS UNIVERSIDAD DISTRITAL,AULA 308,CALLE 40
25425,9919,SABADO,POR ASIGNAR,ALEJANDRO SUAREZ COPETE,SEGUNDA LENGUA I - NATIVA 1,105-54,16-17,0,INSTITUTO DE LENGUAS UNIVERSIDAD DISTRITAL,AULA 308,CALLE 40


In [13]:
def agrupar_df(df):
    for sede, df_sede in df.groupby("Sede"):
        yield {
            "Sede": sede,    
            "Proyectos_curriculares": list(agrupar_proyectos(df_sede))
        }

def agrupar_proyectos(df_sede):
    for proyecto, df_proyecto in df_sede.groupby("Proyecto_curricular"):
        yield {
            "Proyecto_curricular": proyecto,
            "espacios": list(agrupar_espacio(df_proyecto)),
        }
    
        
def agrupar_espacio(df_proyecto):
    for (esp,cod), df_grp in df_proyecto.groupby(["Espacio_Academico","Cod"]):
        yield {
            "Espacio_Academico": esp,
            "Cod": int(cod),
            #"Grp": grupo,
            "grupos": list(agrupar_grupo(df_grp)),
        }
        
def agrupar_grupo(df_grp):
    for (grupo,ins,doc), df_esp in df_grp.groupby(["Grp","INSCRITOS","Docente"]):
        yield {
            "Grp": grupo,
            #"Espacio_Academico": esp,
            "Docente": doc,
            "INSCRITOS": int(ins),
            "lugar": list(agrupar_lugar(df_esp)),
        }

def agrupar_lugar(df_esp):
    for (dia,edd,salon), df_lug in df_esp.groupby(["Dia","Edificio","Salon"]):
        yield {
            "Dia": dia,            
            "Edificio": edd,
            "Salon": salon,
            "horas": list(agrupar_datos(df_lug)),
        }

def agrupar_datos(df_lug):
    nuevos = df_lug.drop(['Dia','Edificio', 'Salon', 'Espacio_Academico', 'Cod', 'INSCRITOS', 'Grp', 'Proyecto_curricular','Docente','Sede'], axis=1)
    for row in nuevos.itertuples():
        yield {
            "Hora": row.Hora,
        }

data = list(agrupar_df(df))

In [14]:
#f = open("pruebas_optimizacion_1.json", 'w+',encoding='utf-8')
f = open("pruebas_optimizacion_2.json", 'w+',encoding='utf-8')
#f.write(json.dumps(data, indent=4, sort_keys=True))
f.write(json.dumps(data,sort_keys=True))
f.close()

# **NO SIRVE!**

In [ ]:
#df.head()
json_dict = {}
json_dict = []
for sede, sede_data in df.groupby('Sede'):
    sede_dict = {}
    sede_dict['Sede'] = sede
    sede_dict = []
    for pr, pr_data in sede_data.groupby('Proyecto_curricular'):
        pr_dict = {}
        pr_dict['Proyecto_curricular'] = pr        
        for grp, grp_data in pr_data.groupby('Grp'):
            pr_dict = []
            grp_dict = {}
            grp_dict['Grp'] = grp            
            for cod, cod_data in grp_data.groupby(['Cod','Espacio_Academico','INSCRITOS']):
                grp_dict = []
                cod_dict = {}
                cod_dict['Cod'] = cod[0]
                cod_dict['Espacio_Academico'] = cod[1]
                cod_dict['INSCRITOS'] = cod[2]                
                for des, des_data in cod_data.groupby(['Edificio','Dia','Salon']):
                    cod_dict = []
                    esp = {}
                    esp['Edificio'] = des[0]
                    esp['Dia'] = des[1]
                    esp['Salon'] = des[2]    
                    esp = []
                    p_data = des_data
                    for hora in p_data.to_dict(orient='records'):
                        print(p_data)

#son_out = dumps(json_dict)
#arsed = json.loads(json_out)
#son.dumps(parsed, indent=4, sort_keys=True)

In [ ]:
data = (df.groupby(['Sede'], as_index=True)
        .apply(lambda x: list(x.groupby(['Proyecto_curricular'], as_index=False)))        
        #.apply(lambda x: list(x.groupby(['Espacio_Academico','Cod','Grp','INSCRITOS'], as_index=False)))
        #.apply(lambda x: x[['Cod','Dia','Docente','Edificio','Espacio_Academico','Grp','Hora','INSCRITOS','Proyecto_curricular','Salon']].to_dict('r'))					                    
        .reset_index()
        .to_json(orient='records'))

In [ ]:
print(json.dumps(json.loads(data), indent=4))

In [ ]:
data = df.groupby('Sede')[['Chain','Food','Healthy']]
       .apply(lambda x: x.set_index('Chain').to_dict(orient='index'))
       .to_dict()
data

In [ ]:
json_out = dumps(data)
parsed = json.loads(json_out)
json.dumps(parsed, indent=4, sort_keys=True)